### 3. Embedding word2vec fasttext
Задача: поиск похожих по эмбеддингам

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0

рабочие ссылки с твитами
https://disk.yandex.ru/i/v5HM-ENiGXZVpQ
https://disk.yandex.ru/i/koR5eMCToCZS2Q

В связи с удалением даных можно взять другой датасет, к примеру

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

#### пример работы с ним 
from corus import load_ods_gazeta
path = 'gazeta.csv.gz'
records = load_ods_gazeta(path)
next(records)
что надо сделать
1. объединить в одну выборку
2. на основе word2vec/fasttext слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)
3. Проверить насколько хорошо работают подходы

для новостных статей задание такое же
1. на основе word2vec/fasttext слоя Embedding реализовать метод поиска ближайших статей
(на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы

In [5]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from string import punctuation
import nltk
import random
import torch
import torch.nn as nn

nltk.download('punkt', quiet=True)

# Отключить вывод ошибок
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import gensim.downloader as api

Посмотрим на данные и объединим в одну выборку

In [2]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

In [3]:
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [4]:
def delete_pattern(text, pattern):
    return re.sub(pattern, ' ', text)

на основе word2vec/fasttext слоя Embedding реализовать метод поиска ближайших твитов (на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)

In [6]:
api.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [8]:
word_vectors = api.load("glove-wiki-gigaword-100")  # загрузим предтренированные вектора слов из gensim-data
# выведем слово наиболее близкое к 'man', 'child' и далекое от 'cat'
result = word_vectors.most_similar(positive=['man', 'child'], negative=['cat'])
print("{}: {:.4f}".format(*result[0]))

woman: 0.7172


In [9]:
# выведем лишнее слово
print(word_vectors.doesnt_match("breakfast cereal dinner lunch".split()))

print(word_vectors.doesnt_match("black green summer brown".split()))

cereal
summer


In [10]:
# определим схожесть между словами
similarity = word_vectors.similarity('child', 'man')
print(similarity)

similarity = word_vectors.similarity('woman', 'man')
print(similarity)

similarity = word_vectors.similarity('cat', 'man')
print(similarity)

0.6682648
0.8323494
0.5261841


In [11]:
# найдем top-5 самых близких слов
result = word_vectors.similar_by_word("man", topn=5)
print(result)

result = word_vectors.similar_by_word("cat", topn=5)
print(result)

result = word_vectors.similar_by_word("child", topn=5)
print(result)

[('woman', 0.8323495388031006), ('boy', 0.7914870977401733), ('one', 0.7788748145103455), ('person', 0.7526815533638), ('another', 0.7522234916687012)]
[('dog', 0.8798074722290039), ('rabbit', 0.7424427270889282), ('cats', 0.732300341129303), ('monkey', 0.7288709878921509), ('pet', 0.719014048576355)]
[('children', 0.8553194999694824), ('mother', 0.7771132588386536), ('parents', 0.7735786437988281), ('girl', 0.7634811997413635), ('woman', 0.7601762413978577)]


In [13]:
!pip install annoy

     ---------------------------------------- 0.0/647.5 kB ? eta -:--:--
     - -------------------------------------- 30.7/647.5 kB ? eta -:--:--
     ------- ------------------------------ 122.9/647.5 kB 1.4 MB/s eta 0:00:01
     --------------- ---------------------- 256.0/647.5 kB 2.3 MB/s eta 0:00:01
     ---------------------- --------------- 389.1/647.5 kB 2.4 MB/s eta 0:00:01
     -------------------------------------- 647.5/647.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for annoy: filename=annoy-1.17.3-cp311-cp311-win_amd64.whl size=52188 sha256=92f1903f11a23aa2a63abe4d587785d1856f9c35168c0bab37cb3433cc9fada4
  Stored in directory: c:\users\инна\appdata\local\pip\cache\wheels\33\e5\58\0a3e34b92bedf09b4c57e37a63ff395ade6f6c1099ba59877c
Successfully built annoy


In [47]:
import string
import pymorphy2

from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import notebook

In [29]:
answers=pd.read_csv('prepared_answers.txt', sep=';')
answers.head(5)

,"__label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^\t__label__2 The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny."


In [30]:
assert True

question = None
written = False

with open("prepared_answers.txt", "w") as fout:
    with open("corpus", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

In [104]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morph.parse(i.lower())[0].normal_form for i in spls]
    #spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [54]:
!pip install pymorphy3

  Obtaining dependency information for pymorphy3 from https://files.pythonhosted.org/packages/d7/f9/ffb9afde503dc6bb2361ea79ceaea18138fbcee32aec4c5d8efa49180753/pymorphy3-1.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for docopt-ng>=0.6 from https://files.pythonhosted.org/packages/6c/4a/c3b77fc1a24510b08918b43a473410c0168f6e657118807015f1f1edceea/docopt_ng-0.9.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
     ---------------------------------------- 0.1/8.4 MB 1.7 MB/s eta 0:00:06
     - -------------------------------------- 0.2/8.4 MB 2.8 MB/s eta 0:00:03
     -- ------------------------------------- 0.5/8.4 MB 3.8 MB/s eta 0:00:03
     ---- ----------------------------------- 1.0/8.4 MB 5.8 MB/s eta 0:00:02
     ---------- ----------------------------- 2.3/8.4 MB 10.3 MB/s eta 0:00:01
     ------------------ --------------------- 3.9/8.4 MB 14.6 MB/s eta 0:00:01
     -------------------------- ------------- 5.

In [105]:
import inspect
import pymorphy3
assert True

#подготовка к обучению

sentences = []

morph = pymorphy3.MorphAnalyzer()
exclude = set(string.punctuation)
c = 0

with open("corpus", "r") as fin:

    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

0it [00:00, ?it/s]

In [106]:
#sentences = [token for tweet in df.text for token in word_tokenize(tweet) if token not in punctuation]
#print(len(sentences))
#sentences[:10]
sentences = [i for i in sentences if len(i) > 2]

In [108]:
sentences[:4]

[['label2',
  'stuning',
  'even',
  'for',
  'the',
  'nongamer',
  'this',
  'sound',
  'track',
  'was',
  'beautiful',
  'it',
  'paints',
  'the',
  'senery',
  'in',
  'your',
  'mind',
  'so',
  'well',
  'i',
  'would',
  'recomend',
  'it',
  'even',
  'to',
  'people',
  'who',
  'hate',
  'vid',
  'game',
  'music',
  'i',
  'have',
  'played',
  'the',
  'game',
  'chrono',
  'cross',
  'but',
  'out',
  'of',
  'all',
  'of',
  'the',
  'games',
  'i',
  'have',
  'ever',
  'played',
  'it',
  'has',
  'the',
  'best',
  'music',
  'it',
  'backs',
  'away',
  'from',
  'crude',
  'keyboarding',
  'and',
  'takes',
  'a',
  'fresher',
  'step',
  'with',
  'grate',
  'guitars',
  'and',
  'soulful',
  'orchestras',
  'it',
  'would',
  'impress',
  'anyone',
  'who',
  'cares',
  'to',
  'listen'],
 ['label2',
  'the',
  'best',
  'soundtrack',
  'ever',
  'to',
  'anything',
  'im',
  'reading',
  'a',
  'lot',
  'of',
  'reviews',
  'saying',
  'that',
  'this',
  'is',


In [109]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1)

In [110]:
modelFT = FastText(sentences=sentences, vector_size=300, min_count=1, window=5, workers=8)

In [119]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

with open("corpus", "r") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[0]
        question = preprocess_txt(spls[0])
        
        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 100000:
            break

w2v_index.build(10)
ft_index.build(10)

0it [00:00, ?it/s]

True

In [120]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

In [130]:
TEXT = "the best track"

In [131]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['__label__2 One of the very best.....: This is one of the very best "rockumentaries" ever! I loved the whole thing, especially the interviews....especially the ones with the ultra-talented producer/singer/song writer/pianist John Simon-my friend. You might consider checking out John\'s own recordings.\n',
 "__label__2 One of 20th Century's Greatest: This book contains supreme beauty and the worst ugliness. One of the greatest works of the 20th century, one all should have a go at.\n",
 '__label__2 The Sky is the Limit: The true story of the Lafayette Escadrille comes to life in MGM\'s action packed war movie "Flyboys" The young american men who fought for the French and became the first fighter pilots during the Great War. It\'s one of the best war movies ever made, a thrill ride into history, the best WWI movie since "All Quiet on the Western Front" It\'s spectacular, a great war film. See "Flyboys"\n',
 '__label__2 True Prediction: On of the best and prophetic novels ever written ab

In [132]:
get_response(TEXT, ft_index, modelFT, index_map)

['__label__2 One of the very best.....: This is one of the very best "rockumentaries" ever! I loved the whole thing, especially the interviews....especially the ones with the ultra-talented producer/singer/song writer/pianist John Simon-my friend. You might consider checking out John\'s own recordings.\n',
 "__label__2 One of 20th Century's Greatest: This book contains supreme beauty and the worst ugliness. One of the greatest works of the 20th century, one all should have a go at.\n",
 '__label__2 The Sky is the Limit: The true story of the Lafayette Escadrille comes to life in MGM\'s action packed war movie "Flyboys" The young american men who fought for the French and became the first fighter pilots during the Great War. It\'s one of the best war movies ever made, a thrill ride into history, the best WWI movie since "All Quiet on the Western Front" It\'s spectacular, a great war film. See "Flyboys"\n',
 '__label__2 good: The concert was her best. Great twists and turns on the concer